In [10]:
import numpy as np
import scipy

In [7]:
# Elementary matrix : In mathematics, an elementary matrix is a matrix which differs from the identity matrix by one single elementary row operation
# 1. Row switching
# 2. Row multiplication
# 3, Row addition

# case1 switch two rows
def switch_rows(A, i, j):
    m,n = A.shape
    E = np.eye(m)
    E[i,i] = 0; E[j,j] = 0
    E[i,j] = 1; E[j,i] = 1
    return E,E @ A

x= np.arange(12).reshape(3,4)
print(x)
print(switch_rows(x, 0, 1)[-1])

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[[ 4.  5.  6.  7.]
 [ 0.  1.  2.  3.]
 [ 8.  9. 10. 11.]]


In [20]:
# permutation matrix to switch rows
def switch_rows_permutation(A, source_list, target_list):
    m,n = A.shape
    P = np.eye(m)
    for i in range(len(source_list)):
        P[source_list[i], source_list[i]] = 0
        P[target_list[i], target_list[i]] = 0
        P[source_list[i], target_list[i]] = 1
        P[target_list[i], source_list[i]] = 1
    return P @ A
x = np.arange(16).reshape(4,4)
print(x)
switch_rows_permutation(x, [0,1],[3,2])

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]


array([[12., 13., 14., 15.],
       [ 8.,  9., 10., 11.],
       [ 4.,  5.,  6.,  7.],
       [ 0.,  1.,  2.,  3.]])

In [4]:
# communication matrix for transpose matrix
# x[i,j] = x[i*n+j] -> 对于Python, matlab按行展开
x = np.arange(12).reshape(4,3)
for i in range(x.shape[0]):
    for j in range(x.shape[1]):
        print(f"{x[i,j]} , {x.flatten()[i*x.shape[1]+j]}")

In [5]:
def transpose_by_commuinication_matrix(x):
    m, n = x.shape
    y = np.zeros((m*n,m*n))
    x_flatten = x.flatten()
    for i in range(m):
        for j in range(i,n):
            if i == j:
              # 如果是对角线元素，则不改变，因此在communication matrix中[i*n+j,i*n+j] = 1
              y[i*n+j,i*n+j] = 1
            else:
              # tranpose x[i,j] and x[j,i]
              # 如果非对角线元素，则交换x[i,j]与x[j,i]- > x[i,j]在flatten中为i*n+j行，x[j,i]在flatten中为j*n+i行，因此是i*n+j,j*n+i=1表示第i*n+j行的元素是原来第j*n+i行，j*n+i,i*n+1=1第j*n+i行的元素是原来第i*n+j行
              y[i*n+j,j*n+i] = 1
              y[j*n+i,i*n+j] = 1
    return y.dot(x_flatten).reshape(n,m),y

In [3]:
x = np.arange(9).reshape(3,3)
print(x)

[[0 1 2]
 [3 4 5]
 [6 7 8]]


In [17]:
# exchange matrix
def echange_matrix(n):
    x = np.zeros((n,n))
    for i in range(n):
        x[i,n-1-i] = 1
    return x
# test exchange matrix
x = echange_matrix(4)
test=np.random.rand(4,4)
print(test)
print(x@test)

[[0.29817673 0.5668421  0.00673651 0.92867801]
 [0.74563511 0.58834465 0.4878704  0.37604444]
 [0.0742193  0.3063073  0.98769045 0.39777847]
 [0.42357609 0.10708826 0.14831382 0.2010552 ]]
[[0.42357609 0.10708826 0.14831382 0.2010552 ]
 [0.0742193  0.3063073  0.98769045 0.39777847]
 [0.74563511 0.58834465 0.4878704  0.37604444]
 [0.29817673 0.5668421  0.00673651 0.92867801]]


In [9]:
x = echange_matrix(4)

In [33]:
# LU decomposition by Scipy
from scipy.linalg import lu
A = np.array([
    [5,2,-4,0],
    [2,1,-2,1],
    [-4,-2,5,0],
    [0,1,0,2]
])
P,L,U = lu(A)
print(U)
P @ A == L @ U

[[ 5.          2.         -4.          0.        ]
 [ 0.          1.          0.          2.        ]
 [ 0.          0.          1.8         0.8       ]
 [ 0.          0.          0.          0.77777778]]


array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [62]:
# LU decomposition by procedure program

import numpy as np
from copy import deepcopy

class naive_lup:
    def __init__(self,A) -> None:
        self.matrix = A
    
    @staticmethod
    def switch_pivot(A,k):
        column_A = A[:,k]
        lower_array = np.abs(column_A[k+1:])
        max_index = np.argmax(lower_array)
        if lower_array[max_index].item() > np.abs(column_A[k]).item():
            P,A = switch_rows(A,k,k+1+max_index)
        else:
            P = np.eye(A.shape[0])
        return P,A


    @staticmethod
    def eliminate_column(A,k):
        m,n = A.shape
        elimination_matrix = np.eye(m)
        for i in range(k+1,m):
            elimination_matrix[i,k] = -A[i,k]/A[k,k]
        return elimination_matrix, elimination_matrix @ A


    def lup(self):
        """
        matrix implementation of LU decomposition with partial pivoting
        """
        m,n = self.matrix.shape
        A = deepcopy(self.matrix)
        P_list = []
        M_list = []
        for i in range(n-1):
            # avoid 0 and interchange rows to geter larger pivots
            P,A = self.switch_pivot(A,i) # bug
            M,A = self.eliminate_column(A,i)
            # print(f"{i}\neliminate column:{A}\n,elimination matrix:{M}")
            P_list.append(P)
            M_list.append(M)
        P = np.eye(m)
        L_inv = np.eye(m)
        for i in range(len(P_list)-1,-1,-1):
            P = P @ P_list[i]
            if i != 0:
                L_inv = L_inv @ M_list[i] @ P_list[i]
            else:
                L_inv = L_inv @ M_list[i]
                for k in range(1,len(P_list)):
                    L_inv = L_inv @ P_list[k]
        L = np.linalg.inv(L_inv)
        return P,L,A
    
    def doolittile(self):
        """
        suppose we don't nedd to interchange any row, and suppose for a square matrix
        """
        m,_ = self.matrix.shape
        A = deepcopy(self.matrix)
        L = np.eye(m)
        U = np.zeros((m,m))
        for i in range(m):
            for j in range(i,m):
                U[i,j] = A[i,j] - L[i,:i] @ U[:i,j] # don't need to dive U[i,i] as the diagnal of L is 1
            for j in range(i+1,m):
                L[j,i] = (A[j,i] - L[j,:i] @ U[:i,i])/U[i,i]
        return L,U  


In [65]:
B = np.array([
    [-2,2,-1],
    [6,-6,7],
    [3,-8,4]
])
x = naive_lup(B)
P,L,U = x.lup()
print(U)
P@B == L@U

[[ 6.         -6.          7.        ]
 [ 0.         -5.          0.5       ]
 [ 0.          0.          1.33333333]]


array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [66]:
x = naive_lup(A)
P,L,U = x.lup()
print(U)
P@A == L@U

[[  5.           2.          -4.           0.        ]
 [  0.          25.          -1.80327869 -13.19672131]
 [  0.           0.           1.08790965   1.33360146]
 [  0.           0.           0.           0.58297994]]


array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True, False,  True],
       [ True, False,  True, False]])

In [63]:
permutation_a = P @ A
d_L,d_U = naive_lup(permutation_a).doolittile()
print(permutation_a == d_L @ d_U)


[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True False  True]
 [ True  True  True  True]]
